In [55]:
from xbbg import blp
import pdblp
import workdays
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

pd.set_option('display.max_columns', 70)
pio.renderers.default = "notebook"



In [58]:
d_from = workdays.workday(datetime.datetime.today(), days=-260*10).strftime("%Y%m%d")
d_to = workdays.workday(datetime.datetime.today(), days=-1).strftime("%Y%m%d")

f = ["px_last","mov_avg_50d","mov_avg_100d","mov_avg_200d","pe_ratio","best_pe_ratio","best_px_bps_ratio","earn_yld"]
T = ["SPX Index","TSEREIT Index"]
T0 = [s.split()[0] for s in T]

dfs = {}
for t, t0 in zip(T, T0):
    df = blp.bdh(t, f, d_from, d_to, Calendar="5D", Fill="P").reset_index()
    df.columns = ["Date"] + f
    df["Date"] = pd.to_datetime(df["Date"])

    dfs[f"{t0}"] = df


In [64]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import pandas as pd

def plot_chart(t, df, *columns, horizontal_lines=None, secondary_y_columns=None, 
               start_date=None, end_date=None, last_days=None, last_months=None, last_years=None):
    """
    チャートをプロットする関数
    
    Parameters:
    -----------
    t : str
        チャートのタイトル
    df : DataFrame
        データフレーム（'Date'列を含む必要がある）
    columns : str
        プロットする列名（複数指定可能）
    horizontal_lines : list, optional
        水平線を引く値のリスト
    secondary_y_columns : list, optional
        右軸に表示する列名のリスト
    start_date : str or datetime, optional
        開始日（'YYYY-MM-DD'形式の文字列またはdatetimeオブジェクト）
    end_date : str or datetime, optional
        終了日（'YYYY-MM-DD'形式の文字列またはdatetimeオブジェクト）
    last_days : int, optional
        直近N日間を表示
    last_months : int, optional
        直近Nヶ月間を表示
    last_years : int, optional
        直近N年間を表示
    """
    
    colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan']
    widths = [2, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5]  # 最初の線だけ太く
    
    hline_colors = ['purple', 'brown', 'pink', 'gray', 'cyan']
    
    # データフレームのコピーを作成（元のデータを変更しないため）
    df_plot = df.copy()
    
    # Date列がdatetime型でない場合は変換
    if not pd.api.types.is_datetime64_any_dtype(df_plot['Date']):
        df_plot['Date'] = pd.to_datetime(df_plot['Date'])
    
    # 期間のフィルタリング
    if last_days is not None:
        end_date = df_plot['Date'].max()
        start_date = end_date - timedelta(days=last_days)
    elif last_months is not None:
        end_date = df_plot['Date'].max()
        start_date = end_date - pd.DateOffset(months=last_months)
    elif last_years is not None:
        end_date = df_plot['Date'].max()
        start_date = end_date - pd.DateOffset(years=last_years)
    else:
        # start_dateとend_dateが文字列の場合はdatetimeに変換
        if start_date is not None and isinstance(start_date, str):
            start_date = pd.to_datetime(start_date)
        if end_date is not None and isinstance(end_date, str):
            end_date = pd.to_datetime(end_date)
    
    # 期間でフィルタリング
    if start_date is not None:
        df_plot = df_plot[df_plot['Date'] >= start_date]
    if end_date is not None:
        df_plot = df_plot[df_plot['Date'] <= end_date]
    
    # データが空の場合の警告
    if len(df_plot) == 0:
        print(f"警告: 指定された期間にデータがありません。")
        return
    
    # 2軸が必要かどうかを判定
    use_secondary_y = secondary_y_columns is not None and len(secondary_y_columns) > 0
    
    if use_secondary_y:
        fig = make_subplots(specs=[[{"secondary_y": True}]])
    else:
        fig = go.Figure()
    
    # データ列をプロット
    for i, column in enumerate(columns):
        if column in df_plot.columns:
            # 右軸に表示するかどうかを判定
            is_secondary = use_secondary_y and column in secondary_y_columns
            
            trace = go.Scatter(
                x=df_plot['Date'],
                y=df_plot[column],
                mode='lines',
                name=column,
                line=dict(
                    color=colors[i % len(colors)],  # 色を順番に割り当て
                    width=widths[i % len(widths)]   # 幅を順番に割り当て
                )
            )
            
            if use_secondary_y:
                fig.add_trace(trace, secondary_y=is_secondary)
            else:
                fig.add_trace(trace)
    
    # 横線を追加
    if horizontal_lines:
        for i, value in enumerate(horizontal_lines):
            if use_secondary_y:
                fig.add_hline(
                    y=value,
                    line_dash="dash",
                    line_color=hline_colors[i % len(hline_colors)],
                    secondary_y=False  # 左軸に横線を追加
                )
            else:
                fig.add_hline(
                    y=value,
                    line_dash="dash",
                    line_color=hline_colors[i % len(hline_colors)]
                )
    
    # タイトルに期間情報を追加
    period_info = ""
    if start_date is not None or end_date is not None:
        start_str = df_plot['Date'].min().strftime('%Y-%m-%d')
        end_str = df_plot['Date'].max().strftime('%Y-%m-%d')
        period_info = f" ({start_str} ~ {end_str})"
    
    fig.update_layout(
        title=t + period_info,
        xaxis_title='',
        hovermode='x unified',
        template='plotly_white',
        width=800,
        height=500,
        font=dict(
            family="Segoe UI, Arial, Helvetica, Verdana, sans-serif"
        )
    )
    
    # X軸のグリッド設定
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    
    if use_secondary_y:
        # 2軸の場合：左軸のグリッドのみ表示、右軸のグリッドは非表示
        fig.update_yaxes(
            showgrid=True, 
            gridwidth=1, 
            gridcolor='LightGray',
            secondary_y=False
        )
        fig.update_yaxes(
            showgrid=False,  # 右軸のグリッドは非表示
            secondary_y=True
        )
        
        # Y軸のタイトル設定（必要に応じて）
        fig.update_yaxes(title_text="", secondary_y=False)
        fig.update_yaxes(title_text="", secondary_y=True)
    else:
        # 1軸の場合
        fig.update_yaxes(
            showgrid=True, 
            gridwidth=1, 
            gridcolor='LightGray',
            title_text=""
        )
    
    fig.show()


# t = "TSEREIT"
# # 全期間
# plot_chart(t, dfs[t], 'px_last', 'mov_avg_50d', 'mov_avg_100d', 'mov_avg_200d')
# # 直近1年間
# plot_chart(t, dfs[t], 'px_last', 'mov_avg_50d', 'mov_avg_100d', 'mov_avg_200d', last_years=1)
# # 直近6ヶ月間
# plot_chart(t, dfs[t], 'px_last', 'mov_avg_50d', 'mov_avg_100d', last_months=6)
# # 直近90日間
# plot_chart(t, dfs[t], 'px_last', 'mov_avg_50d', last_days=90)
# # 特定の期間を指定
# plot_chart(t, dfs[t], 'px_last', 'mov_avg_100d', 
#           start_date='2023-01-01', end_date='2023-12-31',
#           horizontal_lines=[1700, 1800])
# # 2軸グラフで直近2年間
# plot_chart(t, dfs[t], 'px_last', 'pe_ratio', 
#           secondary_y_columns=['pe_ratio'],
#           last_years=2)

In [71]:
t = "SPX"
plot_chart(t, dfs[t], 'px_last', 'mov_avg_50d', 'mov_avg_100d', 'mov_avg_200d', last_years=1)
plot_chart(t, dfs[t], 'px_last', 'pe_ratio', 
          secondary_y_columns=['pe_ratio'],
          last_years=1)


t = "TSEREIT"
plot_chart(t, dfs[t], 'px_last', 'mov_avg_50d', 'mov_avg_100d', 'mov_avg_200d', last_years=1,horizontal_lines=[1800])
plot_chart(t, dfs[t], 'px_last', 'pe_ratio', 
          secondary_y_columns=['pe_ratio'],
          last_years=1)

